In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import os
import json
import re

In [2]:
base_url = "https://#Some_Website"
base_folder ="সৈয়দ-মুজতবা-আলী"

In [3]:
source = requests.get(base_url).text
soup = BeautifulSoup(source,'lxml')

In [4]:
ol= soup.find(class_='archive-description taxonomy-archive-description').ol
main_category = {}

for i in ol:
    if str(type(i))=="<class 'bs4.element.NavigableString'>":
        continue
    try:
        item = i.findChildren()[0]
        main_category[item.text] = item['href']
    except:
        pass

In [5]:
for key,value in main_category.items():
    value = "#_url_of_the_book" # comment out to see actual url
    print(key,'\t',value)

অবিশ্বাস্য (উপন্যাস) 	 #_url_of_the_book
অসি রায়ের গপ্‌পো 	 #_url_of_the_book
কত না অশ্রুজল 	 #_url_of_the_book
চতুরঙ্গ 	 #_url_of_the_book
চাচা কাহিনী 	 #_url_of_the_book
জলে-ডাঙায় 	 #_url_of_the_book
টুনি মেম 	 #_url_of_the_book
তুলনাহীনা 	 #_url_of_the_book
দু-হারা 	 #_url_of_the_book
দেশে বিদেশে 	 #_url_of_the_book
দ্বন্দ্বমধুর (১৯৫৮) 	 #_url_of_the_book
ধূপছায়া 	 #_url_of_the_book
পঞ্চতন্ত্র – ১ম পর্ব 	 #_url_of_the_book
পঞ্চতন্ত্র – ২য় পর্ব 	 #_url_of_the_book
প্রেম 	 #_url_of_the_book
বড়বাবু 	 #_url_of_the_book
ভবঘুরে ও অন্যান্য 	 #_url_of_the_book
মুসাফির 	 #_url_of_the_book
ময়ূরকণ্ঠী (১৯৫২) 	 #_url_of_the_book
রচনাবলি ১০ (দশম খণ্ড) 	 #_url_of_the_book
রচনাবলি ১১ (একাদশ খণ্ড) 	 #_url_of_the_book
রচনাবলি ৮ (অষ্টম খণ্ড) 	 #_url_of_the_book
রচনাবলি ৯ (নবম খণ্ড) 	 #_url_of_the_book
রাজা উজির 	 #_url_of_the_book
শবনম (উপন্যাস) 	 #_url_of_the_book
শহর-ইয়ার 	 #_url_of_the_book
হিটলার 	 #_url_of_the_book


In [6]:
from clean_text import clean_text

def extract_text_from_section(text_link):
    src = requests.get(text_link).text
    soup = BeautifulSoup(src,'lxml')
    txt = soup.find(class_='entry-content entry-content-single').text # or use get_text(separator="\n\n",strip=True)
    return txt

def scrape_all_sections_and_write_file(book_name,sections,parent_directory):
    parent_directory = parent_directory
    
    # book_name_Example = 'চাচা কাহিনী'
    file_name = os.path.join(parent_directory,f'{book_name}.json')
    if os.path.isfile(file_name): 
        return
    
    book_text = ""
    for text_link in sections.values():
        # print(text_link)
        txt = extract_text_from_section(text_link)
        book_text = book_text + txt + '\n\n'
    
    data = re.split("\n", book_text)
    temp = []
    for _data in data:
        temp.extend(_data.split(" "))
    data = temp

    wordList = data
    # uncomment to apply data cleaning
    # wordList = clean_text(data)

    try:
        # Save as text
        # with open(file_name, 'w',encoding="utf-8") as f:
        #     f.write(book_text)
        
        # Save as json 
        with open(file_name, "w", encoding='utf=8') as outfile:
            book_name = book_name.split("(")[0] # 
            json.dump({book_name:wordList}, outfile, ensure_ascii=False)

    except Exception as e:
        print(e)
        if os.path.exists(file_name):
            os.remove(file_name)


def go_inside_category(book_name,book_link,flag=0):
    
    # print(category_name,category_link)

    parent_directory = 'G:\Web_Scrape' #create this folder yourself
    directory = book_name
    path = os.path.join(parent_directory,directory)
    print(book_name) #path
    if not os.path.exists(path):
        os.mkdir(path)

    src = requests.get(book_link).text
    soup2 = BeautifulSoup(src,'lxml')

    ol2= soup2.find_all(class_='entry-title-link')

    sections = {}
    for i in ol2:
        try:
            item = i
            sections[item.text] = item['href']
        except:
            pass
    
    pagination = soup2.find(attrs ={"role" : "navigation"})
    if pagination and pagination.find(class_="active") and pagination.find(class_="pagination-next"):
        next_url=pagination.find(class_="pagination-next").find('a')['href']
        new_sections=go_inside_category(book_name,next_url,flag=1)
        for key in new_sections:
            sections[key]=new_sections[key]
    
    if flag==1:
        return sections

    for key,value in sections.items():
        value = "#_url_of_the_book" # comment out to see actual url
        print(key,'\t',value)

    print(len(sections),"Sections ...",end='\n\n')

    for section_name,section_link in sections.items():
        scrape_all_sections_and_write_file(book_name,sections,path)
        
        #uncomment to work only one book
        # break 

In [7]:
for category_name, category_link in main_category.items():

    try:
        go_inside_category(category_name,category_link)
    except TypeError as e:
        print(f"{category_name} was not scraped, {e}")

    # break #break after 1 iteration

অবিশ্বাস্য (উপন্যাস)
১. মধুগঞ্জ মহকুমা শহর 	 #_url_of_the_book
২. বেকন আণ্ডা বজর্ন 	 #_url_of_the_book
৩. খ্রীষ্টালয় থেকে সদ্যাগত 	 #_url_of_the_book
৪. সত্যকার শিক্ষিত লোক 	 #_url_of_the_book
4 Sections ...

অসি রায়ের গপ্‌পো
0 Sections ...

কত না অশ্রুজল
অল-মসজিদ-উল-আকসা 	 #_url_of_the_book
আধুনিকের আত্মহত্যা 	 #_url_of_the_book
আন্ ফ্রাঙ্ক 	 #_url_of_the_book
কত না অশ্রুজল 	 #_url_of_the_book
চুম্বন 	 #_url_of_the_book
ত্রিমূর্তি 	 #_url_of_the_book
দর্পণ 	 #_url_of_the_book
দ্বন্দ্ব-পুরাণ 	 #_url_of_the_book
ধন্য অবাঙালি! 	 #_url_of_the_book
নট গিলটি 	 #_url_of_the_book
ন্যাকামো 	 #_url_of_the_book
বনে ভূত না মনে ভূত 	 #_url_of_the_book
বিশ্বভারতী প্রাগ 	 #_url_of_the_book
ব্রেন-ড্রেন 	 #_url_of_the_book
ভরাডুবি (আন্ ফ্রাঙ্ক) 	 #_url_of_the_book
মরহুম অধ্যাপক ডক্টর আব্দুল হাই 	 #_url_of_the_book
মাভৈঃ 	 #_url_of_the_book
রহস্য লহরি 	 #_url_of_the_book
রাবাৎ-ইনসল্ট 	 #_url_of_the_book
সিংহ-মূষিক কাহিনী 	 #_url_of_the_book
স্পাই 	 #_url_of_the_book
হিটলারের শেষ প্রেম 	 #_url_of_the_b